In [44]:
with open("twitter2015/train.txt","r") as f:
    num = 0
    data = []
    sample = {}
    words = []
    tags = []
    
    for line in f:
        num +=1
        line = line.strip()
        if line:
            # print(len(line),line)
            if line.startswith("IMGID:"):
                img_id = line[len("IMGID:"):]+".jpg"
                sample = {"img_id":img_id}
                words = []
                tags = []
            elif line.startswith("http:") or line.startswith("https:"):
                continue
            else:    
                parts = line.split("\t")
                words.append(parts[0])
                tags.append(parts[1])
        else:
            if sample:
                text = " ".join(words)
                ents = []
                name = ''
                tag = ''
                pos = [0,0]
                
                for i in range(len(words)):
                    if tags[i].startswith("B-"):
                        if name:
                            ents.append({"name":name,"pos":pos,"tag":tag})
                            name = ''
                            pos = [0,0]
                            
                        tag = tags[i][len("B-"):]
                        name = words[i]
                        pos = [i,i+1]
                    elif tags[i].startswith("I-"):
                        if name == '':
                            continue
                        name += " " +words[i]
                        pos[1] = i+1
                
                if name:
                    ents.append({"name":name,"pos":pos,"tag":tag})
                
                sample["text"] = text 
                sample["ents"] = ents
                data.append(sample)
                sample = {}
            else:
                print("error")     
        # if num >30:
        #     break

In [45]:
len(data)

4000

In [46]:
import json
# 将列表写入JSON文件
with open("twitter15_new/train.json", 'w') as json_file:
    json.dump(data, json_file,indent=4)

json_file.close()

In [47]:
with open("twitter2015/test.txt","r") as f:
    data = []
    sample = {}
    words = []
    tags = []
    
    for line in f:
        line = line.strip()
        if line:
            # print(len(line),line)
            if line.startswith("IMGID:"):
                img_id = line[len("IMGID:"):]+".jpg"
                sample = {"img_id":img_id}
                words = []
                tags = []
            elif line.startswith("http:") or line.startswith("https:"):
                continue
            else:
                parts = line.split("\t")
                words.append(parts[0])
                tags.append(parts[1])
        else:
            if sample:
                text = " ".join(words)
                ents = []
                name = ''
                tag = ''
                pos = [0,0]
                
                for i in range(len(words)):
                    if tags[i].startswith("B-"):
                        if name:
                            ents.append({"name":name,"pos":pos,"tag":tag})
                            name = ''
                            pos = [0,0]
                            
                        tag = tags[i][len("B-"):]
                        name = words[i]
                        pos = [i,i+1]
                    elif tags[i].startswith("I-"):
                        if name == '':
                            continue
                        name += " " +words[i]
                        pos[1] = i+1
                
                if name:
                    ents.append({"name":name,"pos":pos,"tag":tag})
                
                sample["text"] = text 
                sample["ents"] = ents
                data.append(sample)
                sample = {}
            else:
                print("error")   
                
                
with open("twitter15_new/test.json", 'w') as json_file:
    json.dump(data, json_file,indent=4)

json_file.close()                

In [48]:
len(data)

3257

In [49]:
with open("twitter2015/valid.txt","r") as f:
    data = []
    sample = {}
    words = []
    tags = []
    
    for line in f:
        line = line.strip()
        if line:
            # print(len(line),line)
            if line.startswith("IMGID:"):
                img_id = line[len("IMGID:"):]+".jpg"
                sample = {"img_id":img_id}
                words = []
                tags = []
            elif line.startswith("http:") or line.startswith("https:"):
                continue
            else:
                parts = line.split("\t")
                words.append(parts[0])
                tags.append(parts[1])
        else:
            if sample:
                text = " ".join(words)
                ents = []
                name = ''
                tag = ''
                pos = [0,0]
                
                for i in range(len(words)):
                    if tags[i].startswith("B-"):
                        if name:
                            ents.append({"name":name,"pos":pos,"tag":tag})
                            name = ''
                            pos = [0,0]
                            
                        tag = tags[i][len("B-"):]
                        name = words[i]
                        pos = [i,i+1]
                    elif tags[i].startswith("I-"):
                        if name == '':
                            continue
                        name += " " +words[i]
                        pos[1] = i+1
                
                if name:
                    ents.append({"name":name,"pos":pos,"tag":tag})
                
                sample["text"] = text 
                sample["ents"] = ents
                data.append(sample)
                sample = {}
            else:
                print("error")   
                
                
with open("twitter15_new/valid.json", 'w') as json_file:
    json.dump(data, json_file,indent=4)

json_file.close()                

In [50]:
len(data)

1000

# 在文件中加上  BLIP 评估的img与text相似度值

In [ ]:
from PIL import Image
import torch
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode
import sys
sys.path.append('../../Blip/')  
from models.blip_itm import blip_itm


def load_demo_image(image_size,img_path,device):
    
    raw_image = Image.open(img_path).convert("RGB") 

    w,h = raw_image.size
    # display(raw_image.resize((w//5,h//5)))
    
    transform = transforms.Compose([
        transforms.Resize((image_size,image_size),interpolation=InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
        ]) 
    image = transform(raw_image).unsqueeze(0).to(device)   
    return image


In [52]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

image_size = 384

model_url = '/home/users/wgx/Mult_m_NRE/multi-joint-ner-re/Blip/MODELS/model_base_retrieval_coco.pth'
    
model = blip_itm(pretrained=model_url, image_size=image_size, vit='base')
model.eval()
model = model.to(device)

load checkpoint from /home/users/wgx/Mult_m_NRE/multi-joint-ner-re/Blip/MODELS/model_base_retrieval_coco.pth


In [53]:
import json 
train_data = json.load(open("twitter15_new/train.json",'r'))

print("训练集数据条数：",len(train_data))
print("每条数据的样式：",train_data[0].keys(),"\n",train_data[0])

new_train_data = []
file_path = "twitter2015_images/"
image_size = 384

num = 0

for td in train_data:
    num+=1
    sample = td
    
    caption = td["text"]
    img_path = file_path + td["img_id"]
    
    try:
        image = load_demo_image(image_size=image_size,img_path=img_path, device=device)
        
        
        itm_output = model(image,caption,match_head='itm')
        itm_score = torch.nn.functional.softmax(itm_output,dim=1)[:,1]
        # print('The image and text is matched with a probability of %.4f'%itm_score)

        itc_score = model(image,caption,match_head='itc')
        # print('The image feature and text feature has a cosine similarity of %.4f'%itc_score)
        
        sample["itm_score"] = itm_score.cpu().detach().item()
        sample["itc_score"] = itc_score.cpu().detach().item()
        
        # print(sample)
        new_train_data.append(sample)
        # break
        if num %100 ==0:
            print(num)
    except:
        print(img_path)
    
with open("twitter15_new2/new_train.json", 'w') as json_file:
    json.dump(new_train_data, json_file,indent=4)
    
json_file.close()

     

训练集数据条数： 4000
每条数据的样式： dict_keys(['img_id', 'text', 'ents']) 
 {'img_id': '1015799.jpg', 'text': 'RT @JayKenMinaj _ : Me outside of where George Zimmerman got shot at . You know God is so good .', 'ents': [{'name': 'George Zimmerman', 'pos': [8, 10], 'tag': 'PER'}]}
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000


In [54]:
len(new_train_data)

4000

In [55]:
import json 
test_data = json.load(open("twitter15_new/test.json",'r'))

print("训练集数据条数：",len(test_data))
print("每条数据的样式：",test_data[0].keys(),"\n",test_data[0])

new_test_data = []
file_path = "twitter2015_images/"
image_size = 384

num = 0

for td in test_data:
    num+=1
    sample = td
    
    caption = td["text"]
    img_path = file_path + td["img_id"]
    try:
        image = load_demo_image(image_size=image_size,img_path=img_path, device=device)
        
        
        itm_output = model(image,caption,match_head='itm')
        itm_score = torch.nn.functional.softmax(itm_output,dim=1)[:,1]
        # print('The image and text is matched with a probability of %.4f'%itm_score)

        itc_score = model(image,caption,match_head='itc')
        # print('The image feature and text feature has a cosine similarity of %.4f'%itc_score)
        
        sample["itm_score"] = itm_score.cpu().detach().item()
        sample["itc_score"] = itc_score.cpu().detach().item()
        
        # print(sample)
        new_test_data.append(sample)
        # break
        if num %100 ==0:
            print(num)
    except:
        print(img_path)
    
with open("twitter15_new2/new_test.json", 'w') as json_file:
    json.dump(new_test_data, json_file,indent=4)
    
json_file.close()

     

训练集数据条数： 3257
每条数据的样式： dict_keys(['img_id', 'text', 'ents']) 
 {'img_id': '62654.jpg', 'text': 'RT @ThePatriot143 : TIMELINE SHOWS HOW CLINTONS TOOK $ 1 . 8 MILLION FROM KEYSTONE PIPELINE INVESTORS #ClintonCash', 'ents': [{'name': 'CLINTONS', 'pos': [6, 7], 'tag': 'ORG'}, {'name': 'KEYSTONE', 'pos': [14, 15], 'tag': 'ORG'}]}
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200


In [56]:
len(new_test_data)

3257

In [57]:
import json 
valid_data = json.load(open("twitter15_new/valid.json",'r'))

print("训练集数据条数：",len(valid_data))
print("每条数据的样式：",valid_data[0].keys(),"\n",valid_data[0])

new_valid_data = []
file_path = "twitter2015_images/"
image_size = 384

num = 0

for td in valid_data:
    num+=1
    sample = td
    
    caption = td["text"]
    img_path = file_path + td["img_id"]
    try:
        image = load_demo_image(image_size=image_size,img_path=img_path, device=device)
        
        
        itm_output = model(image,caption,match_head='itm')
        itm_score = torch.nn.functional.softmax(itm_output,dim=1)[:,1]
        # print('The image and text is matched with a probability of %.4f'%itm_score)

        itc_score = model(image,caption,match_head='itc')
        # print('The image feature and text feature has a cosine similarity of %.4f'%itc_score)
        
        sample["itm_score"] = itm_score.cpu().detach().item()
        sample["itc_score"] = itc_score.cpu().detach().item()
        
        # print(sample)
        new_valid_data.append(sample)
        # break
        if num %100 ==0:
            print(num)
    except:
        print(img_path)
    
with open("twitter15_new2/new_valid.json", 'w') as json_file:
    json.dump(new_valid_data, json_file,indent=4)
    
json_file.close()

     

训练集数据条数： 1000
每条数据的样式： dict_keys(['img_id', 'text', 'ents']) 
 {'img_id': '32977.jpg', 'text': 'RT @jonathanchait : How post-Katrina New Orleans proved urban education reform can work', 'ents': [{'name': 'New Orleans', 'pos': [5, 7], 'tag': 'LOC'}]}
100
200
300
400
500
600
700
800
900
1000


In [58]:
len(new_valid_data)

1000